In [126]:
import numpy as np
import pandas as pd
import os
import sys
from math import frexp, ldexp
import math
import time
from condTE_utils import CondTEUtils as CTU

## Data loading

In [127]:
# load data
datapath = '../Firearms'
data_original = pd.read_excel(os.path.join(datapath,'summary_data.xlsx'))
data_original_state = pd.read_excel(os.path.join(datapath,'background_checks_processed.xlsx'))

BC = data_original_state.values[:,1:]
State_names = data_original_state.columns.values[1:]
MS = data_original['Mass shooting'].values
MO = data_original['Articles ("firearm laws and regulations")'].values

# symbolic
XBC = (np.diff(BC,axis=0) > 0) * 1
XMS = (MS[:-1] > 0)*1
XMO = (np.diff(MO,axis=0) > 0) * 1
Nt = len(XMO)

## Experiment

In [128]:
nsurr = 50000

In [129]:
condTeResults = {state: CTU.cal_transfer_entropy(XBC[:,i],XMO,XMS) for i, state in enumerate(State_names)}
condTeResults

{'Alabama': 0.045687806191075264,
 'Alaska': 0.005138575170988702,
 'Arizona': 0.03595264422458033,
 'Arkansas': 0.046056471482139205,
 'California': 0.002854575356465716,
 'Colorado': 0.010607858676976889,
 'Connecticut': 0.042180243072701185,
 'Delaware': 0.02315482468236363,
 'Florida': 0.03944052019191702,
 'Georgia': 0.024666665324504233,
 'Hawaii': 0.00019786791503317858,
 'Idaho': 0.040983791300772676,
 'Illinois': 0.015348527018638425,
 'Indiana': 0.006336937144982624,
 'Iowa': 0.03247229617662972,
 'Kansas': 0.04002496617617224,
 'Kentucky': 0.017799051419385314,
 'Louisiana': 0.011968793170482162,
 'Maine': 0.0029351940744368754,
 'Maryland': 0.015162911346103591,
 'Massachusetts': 0.008015226448848445,
 'Michigan': 0.015439007374295599,
 'Minnesota': 0.011848077096684229,
 'Mississippi': 0.009162243215014687,
 'Missouri': 0.02605231103965444,
 'Montana': 0.005076153406202266,
 'Nebraska': 0.029730367087961174,
 'Nevada': 0.036466619674146106,
 'New Hampshire': 0.011049997154

In [130]:
surrTEResults = {}
for i, state in enumerate(State_names):
    b = time.time()
    surrTEResults[state] = CTU.surrogate_cond_transfer_entropy(XBC[:,i],XMO,XMS,nsurr)
    e = time.time()
    s = 'State {} finished. It takes {:5.3} seconds. ..... {} / {}'.format(state, e - b, i+1, 50)
    print(s)

State Alabama finished. It takes 2.66e+02 seconds. ..... 1 / 50
State Alaska finished. It takes 2.64e+02 seconds. ..... 2 / 50
State Arizona finished. It takes 2.67e+02 seconds. ..... 3 / 50
State Arkansas finished. It takes 2.72e+02 seconds. ..... 4 / 50
State California finished. It takes 2.74e+02 seconds. ..... 5 / 50
State Colorado finished. It takes 3.18e+02 seconds. ..... 6 / 50
State Connecticut finished. It takes 2.83e+02 seconds. ..... 7 / 50
State Delaware finished. It takes 2.56e+02 seconds. ..... 8 / 50
State Florida finished. It takes 2.53e+02 seconds. ..... 9 / 50
State Georgia finished. It takes 2.52e+02 seconds. ..... 10 / 50
State Hawaii finished. It takes 2.5e+02 seconds. ..... 11 / 50
State Idaho finished. It takes 2.53e+02 seconds. ..... 12 / 50
State Illinois finished. It takes 2.52e+02 seconds. ..... 13 / 50
State Indiana finished. It takes 2.52e+02 seconds. ..... 14 / 50
State Iowa finished. It takes 2.52e+02 seconds. ..... 15 / 50
State Kansas finished. It takes

In [131]:
p_values = {}
ninetyfive_tab = {}
for i, state in enumerate(State_names):
    count_aux_sum = np.sum(surrTEResults[state] > condTeResults[state])
    p_values[state] = (count_aux_sum + 1) / (nsurr + 1)
    ninetyfive_tab[state] = np.quantile(surrTEResults[state], 0.95)
p_values

{'Alabama': 0.008139837203255935,
 'Alaska': 0.8173636527269454,
 'Arizona': 0.03013939721205576,
 'Arkansas': 0.007859842803143937,
 'California': 0.9290414191716165,
 'Colorado': 0.5243495130097398,
 'Connecticut': 0.014039719205615888,
 'Delaware': 0.14573708525829485,
 'Florida': 0.019739605207895843,
 'Georgia': 0.11745765084698306,
 'Hawaii': 0.74852502949941,
 'Idaho': 0.01611967760644787,
 'Illinois': 0.3290934181316374,
 'Indiana': 0.753584928301434,
 'Iowa': 0.052058958820823585,
 'Kansas': 0.017819643607127858,
 'Kentucky': 0.25657486850262995,
 'Louisiana': 0.46997060058798823,
 'Maine': 0.9271614567708646,
 'Maryland': 0.33433331333373334,
 'Massachusetts': 0.6650066998660027,
 'Michigan': 0.3288134237315254,
 'Minnesota': 0.46849063018739623,
 'Mississippi': 0.6042879142417151,
 'Missouri': 0.0984980300393992,
 'Montana': 0.8221235575288495,
 'Nebraska': 0.06779864402711945,
 'Nevada': 0.02847943041139177,
 'New Hampshire': 0.508629827403452,
 'New Jersey': 0.339153216935